### This Workbook will calculate Portfolio Beta as a measure of risk, and Portfolio risk adjusted excess performance (Alpha) over the SPX/GSPC index. 

#### NOTE:

Equity Security selection was biased in the following ways:<br>
1. Selected companies are industry leaders and are among the top 200 US listed companies by Market Capitalisation.<br>
2. These Shares enjoy disproportionately large and continous flows through target date funds and passive/ETF market cap. weighted investment vehicles. At minimum, these mechanisms act as strong share price supports, even in down markets. <br>
3. In many cases, these companies are high margin businesses in respect to their peer group. <br>
4. Selected companies enjoy strong brand recognition, superior products and overall competitive positioning in relation to peers. <br>
4. These businesses have the capacity and do conduct share buybacks in larger proportions than the average Wilshire5000 or S&P500 constituent, boosting shareholder returns.<br>

Furthermore, company financials were not consulted, avoidance of sector concentration was taken into account, and there was no consideration given to forecasting technical levels, interest rates, earnings, valuations, or any other company specific or macro/systematic variable.<br>

__Model Limitations:__ 

This Workbook utilises principles behing the Capital Asset Pricing Model (CAPM). CAPM builds upon methodology proposed by the Modern Portfolio Theory, and as such suffers from some of the same pitfalls. Refer to commentary in Workbook 3 for more detail.

Furthermore, estimates of the future risk-free rate and the expected return on the market are also subject to error. For example, at the end of 2021, rate futures markets (Fed Funds, 3M Libor) significantly underestimated where reference/risk-free rates would be at the end of 2022.  

#### Import Libraries

In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd

import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline
import seaborn as sns # Plotting

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance
import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

# Data API
from alpha_vantage.timeseries import TimeSeries  
# API key from alpha vantage
key = open(r"C:/Users/User/Documents/PM Tools/US Share Market Data/Alpha_vantage_api_key.txt").read()

#### Constants

In [3]:
# Define path to files
PATH = (r"C:/Users/User/Documents/PM Tools/US Share Market Data/Demonstrator Portfolio/")
t_days = 252 # average number of trading days in a year

S_YEAR = 2017
S_MONTH = 1
S_DAY = 1
S_DATE_STR = f"{S_YEAR}-{S_MONTH}-{S_DAY}" #string representation of start date
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY) #datetime object representation of start date

E_YEAR = 2021
E_MONTH = 12
E_DAY = 31
E_DATE_STR = f"{E_YEAR}-{E_MONTH}-{E_DAY}" #string representation of end date
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY) #datetime object representation of end date

risk_free_rate = 0.0152 # 10 year UST yield at the end of 2021 as risk free proxy 

#### Save Dataframe to CSV

In [4]:
def save_df_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

#### Return Dataframe from a CSV

In [5]:
def get_df_from_csv(ticker):
    try:
        # read csv and select the date column for index
        df = pd.read_csv(PATH + ticker + '.csv', index_col='date', parse_dates=True)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

#### Add Daily Return to Dataframe

In [6]:
# Daily Rate of Return = (Ending Price - Beginning Price) / Beginning Price OR (Ending Price / Beginning Price) - 1
def add_daily_return_to_df(df, ticker):
    df.rename(columns = {'5. adjusted close' : 'Adj. Close'}, inplace=True)
    df['daily_return'] = (df['Adj. Close'] / df['Adj. Close'].shift(-1)) - 1 
    # save_df_to_csv(df, ticker); Optional functionality
    return df

#### Merge Multiple Assets by Column Name

In [7]:
# Merge shares by common column name and return dataframe
def merge_df_by_column_name(column_name, sdate, edate, *tickers):
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        # check if dataframe has duplicate indexes
        if not df.index.is_unique:
            # delete duplicates 
            df = df.loc[~df.index.duplicated(), :]
        # get data between defined dates. 
        mask = (df.index >= sdate) & (df.index <= edate)
        mult_df[x] = df.loc[mask][column_name]
        
    return mult_df

#### Download share price data for Portfolio Shares and add daily return column

In [9]:
# refer to workbook 3 for security selection and individual weights
portfolio_list = ["MSFT", "WMT", "UNH", "JNJ", "XOM", "JPM", "PG", "TMO", "DHR", "VZ", 
                  "PM", "HON", "CAT", "CVS", "MCK", "CDNS", "WMB", "DD", "K"]
shares_not_downloaded = [] #list for shares not downloaded

In [10]:
# download daily adjusted prices for 19 portfolio shares
portfolio_folder = (r"C:/Users/User/Documents/PM Tools/US Share Market Data/Demonstrator Portfolio/")
def download_shares_data_to_csv(folder, ticker):
    try:
        print("Get data for:", ticker)
        api_call = TimeSeries(key, output_format='pandas') #api call (alpha vantage)
        df = api_call.get_daily_adjusted(ticker, outputsize='full') # get daily adjusted prices and save to data frame
        time.sleep(1) #pause for 1 second between calls to cut down on missing data 
        df[0].to_csv(folder + ticker + ".csv") # save ticker data from df to csv
        # index at 0 returns the dataframe. Otherwise, it would output a tuple and an error.
    except Exception as ex:
        shares_not_downloaded.append(ticker) #append list of shares not downloaded
        print("Couldn't get data for:", ticker)  

In [11]:
# initialise download
for x in range(19):
    download_shares_data_to_csv(portfolio_folder, portfolio_list[x])
print("Finished")

Get data for: MSFT
Get data for: WMT
Get data for: UNH
Get data for: JNJ
Get data for: XOM
Get data for: JPM
Get data for: PG
Get data for: TMO
Get data for: DHR
Get data for: VZ
Get data for: PM
Get data for: HON
Get data for: CAT
Get data for: CVS
Get data for: MCK
Get data for: CDNS
Get data for: WMB
Get data for: DD
Get data for: K
Finished


In [12]:
# add daily returns column to portfolio shares and save to csv
for ticker in portfolio_list:
    print("Working on:", ticker)
    shares_df = get_df_from_csv(ticker)
    add_daily_return_to_df(shares_df, ticker)
    save_df_to_csv(shares_df, ticker) 

Working on: MSFT
Working on: WMT
Working on: UNH
Working on: JNJ
Working on: XOM
Working on: JPM
Working on: PG
Working on: TMO
Working on: DHR
Working on: VZ
Working on: PM
Working on: HON
Working on: CAT
Working on: CVS
Working on: MCK
Working on: CDNS
Working on: WMB
Working on: DD
Working on: K


#### Get S&P 500 Data

SPDR S&P 500 ETF (ticker: SPY) used as a trading proxy for ^GSPC/SPX index.

In [13]:
# download and save SPY data to csv 
SP_ticker = 'SPY' 

sp_df = pd.DataFrame()

def save_index_data_to_csv(folder, ticker):
    try:
        print("Get data for:", ticker)
        api_call = TimeSeries(key, output_format='pandas') #api call (alpha vantage)
        df = api_call.get_daily_adjusted(ticker, outputsize='full') # get daily adjusted prices and save to data frame
        df[0].to_csv(folder + ticker + ".csv") # save ticker data from df to csv
        # index at 0 returns the dataframe. Otherwise, it would output a tuple and an error.
    except Exception as ex:
        print("Couldn't get data for:", ticker)  

In [14]:
save_index_data_to_csv(PATH, SP_ticker)

Get data for: SPY


In [15]:
# return dataframe for SPY
sp_df = get_df_from_csv('SPY')
sp_df

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2023-01-12,396.670000,398.485000,392.420000,396.960000,396.960000,90157699.0,0.0,1.0
2023-01-11,392.230000,395.600000,391.380000,395.520000,395.520000,68881080.0,0.0,1.0
2023-01-10,387.250000,390.650000,386.270000,390.580000,390.580000,65358094.0,0.0,1.0
2023-01-09,390.370000,393.700000,387.670000,387.860000,387.860000,73978071.0,0.0,1.0
2023-01-06,382.610000,389.250000,379.412700,388.080000,388.080000,104189603.0,0.0,1.0
...,...,...,...,...,...,...,...,...
1999-11-05,138.625000,139.109299,136.781204,137.875000,90.068563,7431500.0,0.0,1.0
1999-11-04,136.750000,137.359299,135.765594,136.531204,89.190712,7907500.0,0.0,1.0
1999-11-03,136.000000,136.375000,135.125000,135.500000,88.517065,7222300.0,0.0,1.0


#### Add daily returns to SPY and save to CSV

In [16]:
add_daily_return_to_df(sp_df, 'SPY')
sp_df

,1. open,2. high,3. low,4. close,Adj. Close,6. volume,7. dividend amount,8. split coefficient,daily_return
date,,,,,,,,,
2023-01-12,396.670000,398.485000,392.420000,396.960000,396.960000,90157699.0,0.0,1.0,0.003641
2023-01-11,392.230000,395.600000,391.380000,395.520000,395.520000,68881080.0,0.0,1.0,0.012648
2023-01-10,387.250000,390.650000,386.270000,390.580000,390.580000,65358094.0,0.0,1.0,0.007013
2023-01-09,390.370000,393.700000,387.670000,387.860000,387.860000,73978071.0,0.0,1.0,-0.000567
2023-01-06,382.610000,389.250000,379.412700,388.080000,388.080000,104189603.0,0.0,1.0,0.022932
...,...,...,...,...,...,...,...,...,...
1999-11-05,138.625000,139.109299,136.781204,137.875000,90.068563,7431500.0,0.0,1.0,0.009842
1999-11-04,136.750000,137.359299,135.765594,136.531204,89.190712,7907500.0,0.0,1.0,0.007610
1999-11-03,136.000000,136.375000,135.125000,135.500000,88.517065,7222300.0,0.0,1.0,0.006734


In [17]:
save_df_to_csv(sp_df, 'SPY')
sp_df

,1. open,2. high,3. low,4. close,Adj. Close,6. volume,7. dividend amount,8. split coefficient,daily_return
date,,,,,,,,,
2023-01-12,396.670000,398.485000,392.420000,396.960000,396.960000,90157699.0,0.0,1.0,0.003641
2023-01-11,392.230000,395.600000,391.380000,395.520000,395.520000,68881080.0,0.0,1.0,0.012648
2023-01-10,387.250000,390.650000,386.270000,390.580000,390.580000,65358094.0,0.0,1.0,0.007013
2023-01-09,390.370000,393.700000,387.670000,387.860000,387.860000,73978071.0,0.0,1.0,-0.000567
2023-01-06,382.610000,389.250000,379.412700,388.080000,388.080000,104189603.0,0.0,1.0,0.022932
...,...,...,...,...,...,...,...,...,...
1999-11-05,138.625000,139.109299,136.781204,137.875000,90.068563,7431500.0,0.0,1.0,0.009842
1999-11-04,136.750000,137.359299,135.765594,136.531204,89.190712,7907500.0,0.0,1.0,0.007610
1999-11-03,136.000000,136.375000,135.125000,135.500000,88.517065,7222300.0,0.0,1.0,0.006734


#### Beta as a measure of Risk

Beta provides the relationship between an individual asset and the overall market (S&P500, Wilshire5000 etc.). Shares with higher Beta suggests higher volatility (risk) in relation to the market. Shares with higher Beta's are expected to outperform the market during periods of positive Share Market returns, but may underperform the benchmark during periods where the market is declining. Beta can be computed for Investment Portfolios. 

Share's Beta ($\beta$) is found by dividing the covariance of the share and the market by the variance of the overall market. It is a measure of systematic risk that can't be diversified away.

$ \beta = \frac{Cov(r_x, r_m)}{\sigma_m^2} $<br>

$ \beta = 1 $ : Share is as risky as the market.<br>
$ \beta < 1 $ : Share is less risky than the market.<br>
$ \beta > 1 $ : Share is more risky than the market.<br>


#### Find Beta for any individual Security 

Calculate Beta over a 5 year period.

In [18]:
msft_df = get_df_from_csv('MSFT')
msft_df

,1. open,2. high,3. low,4. close,Adj. Close,6. volume,7. dividend amount,8. split coefficient,daily_return
date,,,,,,,,,
2023-01-12,235.260,239.9000,233.56,238.51,238.510000,27269486.0,0.0,1.0,0.011621
2023-01-11,231.290,235.9500,231.11,235.77,235.770000,28669331.0,0.0,1.0,0.030238
2023-01-10,227.755,231.3100,227.33,228.85,228.850000,27033881.0,0.0,1.0,0.007617
2023-01-09,226.450,231.2366,226.41,227.12,227.120000,27369784.0,0.0,1.0,0.009736
2023-01-06,223.000,225.7601,219.35,224.93,224.930000,43613574.0,0.0,1.0,0.011785
...,...,...,...,...,...,...,...,...,...
1999-11-05,91.810,92.8700,90.50,91.56,28.692677,35083700.0,0.0,1.0,-0.002071
1999-11-04,92.310,92.7500,90.31,91.75,28.752219,27119700.0,0.0,1.0,-0.002717
1999-11-03,92.940,93.5000,91.50,92.00,28.830563,22258500.0,0.0,1.0,-0.006050


In [19]:
# Function that returns Beta
def get_beta(ticker):
    # tickers: SPY and Share analysed
    ticker_list = ['SPY']
    # append to list ticker we want beta for
    ticker_list.append(ticker) 

    mult_df = merge_df_by_column_name('daily_return', '2017-01-03', '2021-12-31', *ticker_list)
    
    # covariance between individual security and the market
    covariance = mult_df.cov() 
    # return covariance at row 0 & column 1
    covariance_vs_market = covariance.iloc[0,1]
    
    # variance for SPY
    sp_variance = mult_df['SPY'].var() 
    
    beta = covariance_vs_market / sp_variance   
    return beta

In [20]:
# Get 5 Year (2017-2021) Beta for MSFT
print("Microsoft Beta :", get_beta('MSFT')) 

Microsoft Beta : 1.2124463795991822


#### Capital Asset Pricing Model (CAPM)

CAPM is a model that derives the theoretical required expected return (i.e., discount rate) for an asset in a market, given the risk-free rate available to investors and the risk of the market as a whole. It takes into account the asset's sensitivity to non-diversifiable or market risk, represented by the quantity beta ($\beta$). CAPM demonstrates how the market should price individual securities in relation to their security risk class, and assists with creating portfolios that match the risk an investor is willing to assume. 

CAPM formula: $ r_i = r_f + \beta_i (r_m - r_f) $

The securities expected return ($r_{i}$) equals the return on a risk-free asset ($r_{f}$) plus, asset's Beta $(\beta_i$) times the market return ($r_{m}$) minus the risk-free asset return ($r_{i}$).<br> ($r_{m} - r_{f}$) is called the Equity Risk Premium, the expected excess return of the market portfolio's expected return over the risk-free rate. Long term equity risk premium for US Markets ranges between 4% to 6%. We will assume a **5% equity risk premium** going forward. 

Once the asset's expected return, $(r_{i})$, is calculated using CAPM, the future cash flows of the asset can be discounted to their present value using this rate to establish the correct price for the asset. A riskier share will have a higher beta and will be discounted at a higher rate; less sensitive shares will have lower betas and be discounted at a lower rate.

In practice, low-beta securities earn a return somewhat higher than CAPM would predict, and high-beta shares earn less than predicted.  



#### Use CAMP formula to calculate Expected Return for individual securities

In [25]:
# calculate the expected return for Microsoft using a 5 year Beta
ri = risk_free_rate + get_beta('MSFT') * 0.05 # 0.05 = 5% equity risk premium => [r(m) - r(f)]
print("Expected return for Microsoft:", ri * 100, "%")

Expected return for Microsoft: 7.582231897995912 %


### Finding Portfolio Beta 


#### Get Stock Prices on Date

In [30]:
def get_prices_on_date(stocks_df, date):
    return stocks_df.loc[pd.DatetimeIndex([date])]['Adj. Close'].item()

#### Return Portfolio Value by Input Date

In [26]:
def get_portfolio_value_by_date(date, shares, tickers):
    portfolio_prices = merge_df_by_column_name('Adj. Close', date, date, *portfolio_list)
    # convert from dataframe to Python list
    portfolio_prices = portfolio_prices.values.tolist()
    # convert a list of lists into a single list
    portfolio_prices = sum(portfolio_prices, [])
    
    # create a list of values by multiplying number of shares by share prices
    values_list = []
    for price, share in zip(portfolio_prices, shares):
        values_list.append(price * share)
    
    return sum(values_list)

#### Get Portfolio Value at Beginning and End of the Year

In [28]:
# Refer to workbook 3 for security selection and individual weights
'''Portfolio shares: "MSFT", "WMT", "UNH", "JNJ", "XOM", "JPM", "PG", "TMO", "DHR", 
                     "VZ", "PM", "HON", "CAT", "CVS", "MCK", "CDNS", "WMB", "DD", "K".'''

# Number of purchased shares for each security using 03 January 2022 open prices. 
# Functions use adjusted close prices. Transaction prices would probably be somewhere in between.
# Assuming no transactions fees.
portfolio_no_shares = [375, 716, 225, 568, 129, 106, 646, 100, 105, 647, 294, 136, 4, 421, 132, 642, 1542, 44, 17]

# Portfolio value at the start of 2022
portfolio_value_start = get_portfolio_value_by_date('2022-01-03', portfolio_no_shares, portfolio_list)
print("Portfolio Value at Start of 2022 : $%2.2f" % (portfolio_value_start))

# Portfolio value at the end of 2022
portfolio_value_end = get_portfolio_value_by_date('2022-12-30', portfolio_no_shares, portfolio_list)
print("Portfolio Value at End of 2022 : $%2.2f" % (portfolio_value_end))

Portfolio Value at Start of 2022 : $980389.76
Portfolio Value at End of 2022 : $952063.82


#### Calculate Return on Investment

In [34]:
# Portfolio Return on Investment (ROI)
roi_portfolio = (portfolio_value_end - portfolio_value_start) / portfolio_value_start
print("Portfolio ROI at the End of 2022 : %2.2f %%" % (roi_portfolio * 100))

# Market (SPY) ROI
sp_df = get_df_from_csv('SPY')
sp_value_start = get_prices_on_date(sp_df, '2022-01-03')
sp_value_end = get_prices_on_date(sp_df, '2022-12-30')
sp_roi = (sp_value_end - sp_value_start) / sp_value_start
print("SPY ROI at the End of 2022 : %2.2f %%" % (sp_roi * 100))

Portfolio ROI at the End of 2022 : -2.89 %
SPY ROI at the End of 2022 : -18.64 %


#### Portfolio Daily Returns

In [38]:
# Multiply the daily price by the number of shares for each security. 
# Sum those values for all shares for each day, giving daily portfolio value.  
# Calculate portfolio daily return and output dataframe.

def get_portfolio_daily_return(sdate, edate, shares, tickers):
    # Merge daily prices for all shares into dataframe
    mult_df = merge_df_by_column_name('Adj. Close',  sdate, edate, *portfolio_list)
    
    # Get number of shares (no. of columns) in portfolio
    num_columns = len(mult_df.columns)
    
    # Multiply each securities daily price by the number of shares
    i = 0
    while i < num_columns:
        mult_df[tickers[i]] = mult_df[tickers[i]].apply(lambda x: x * shares[i])
        i += 1
        
    # Create new column 'Total' and calculate portfolio's daily value 
    mult_df['Total'] = mult_df.iloc[:, 0:num_columns].sum(axis=1) # get all values (:, 0:num_cols) 
    
    # Add column and calculate portfolio daily return
    mult_df['daily_return'] = (mult_df['Total'] / mult_df['Total'].shift(-1)) - 1
    
    return mult_df

In [39]:
portfolio_df = get_portfolio_daily_return('2022-01-03', '2022-12-30', portfolio_no_shares, portfolio_list)
portfolio_df

,MSFT,WMT,UNH,JNJ,XOM,JPM,PG,TMO,DHR,VZ,...,HON,CAT,CVS,MCK,CDNS,WMB,DD,K,Total,daily_return
date,,,,,,,,,,,,,,,,,,,,,
2022-12-30,89932.500000,101521.640000,119290.500000,100337.200000,14228.700000,14110.349175,97907.760000,55069.000000,27869.100000,25095.978726,...,29144.800000,958.240000,39232.990000,49515.840000,103130.88,50731.800000,3019.720000,1211.080000,952063.817901,-0.003540
2022-12-29,90378.750000,101779.400000,119223.000000,100854.080000,14086.800000,14017.753297,98573.140000,55701.000000,28019.250000,25006.805197,...,29206.000000,960.800000,39342.450000,49515.840000,103959.06,50839.740000,3017.520000,1214.990000,955446.238494,0.010633
2022-12-28,87948.750000,101163.640000,118901.250000,100342.880000,13981.020000,13937.784129,98166.160000,54341.000000,27205.212636,24720.175999,...,28857.840000,957.520000,39051.960000,49831.320000,101654.28,50454.240000,2983.200000,1219.240000,945393.832763,-0.010069
2022-12-27,88860.000000,102967.960000,119697.750000,100780.240000,14214.510000,13862.023865,99451.700000,54544.000000,27366.761288,25000.435659,...,29225.040000,972.560000,39161.420000,50185.080000,102752.10,51518.220000,3026.320000,1235.390000,955009.430812,0.000751
2022-12-23,89523.750000,102939.320000,119544.750000,100808.640000,14019.720000,13813.621474,98592.520000,54068.000000,27259.761531,24465.394489,...,29086.320000,959.480000,39388.760000,50644.440000,103650.90,51379.440000,3024.120000,1227.060000,954292.857494,0.004697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-07,116658.497655,102137.958285,101870.189372,96280.673433,8548.932687,17149.075796,102576.554805,60573.482387,30929.427400,33136.787619,...,28753.035096,876.312206,42888.767566,32770.417948,106584.84,41048.313616,3590.584045,1094.958822,955105.043139,-0.002777
2022-01-06,116599.061406,101172.198033,104324.758916,94996.562123,8479.429169,16980.826907,102633.282653,61903.550617,31792.442843,32460.526647,...,28096.013251,867.712862,42230.143628,32094.698648,111143.04,40212.985563,3538.790550,1084.436696,957764.440671,-0.006867
2022-01-05,117527.752796,101454.171829,108776.304042,95323.124913,8284.571092,16802.318939,103503.109650,61715.964100,31936.801789,32617.515801,...,28127.998088,858.957167,43555.624303,32448.959252,110822.04,39275.073363,3530.158301,1079.668857,964387.105315,-0.005586


#### Find Portfolio Beta 

Calculate Portfolio Beta and Alpha for 2022.  

 

In [43]:
def portfolio_beta(portfolio_df, sdate, edate):
    # dataframe for SPY and demonstrator portfolio
    mult_df = pd.DataFrame()
    
    # mask of required date range for portfolio
    portfolio_mask = (portfolio_df.index >= sdate) & (portfolio_df.index <= edate)
    
    # get SPY dataframe
    sp_df = get_df_from_csv('SPY')
    
    # mask of required date range for SPY
    sp_mask = (sp_df.index >= sdate) & (sp_df.index <= edate)
    
    # add SPY daily return to merged dataframe
    mult_df['SPY'] = sp_df.loc[sp_mask]['daily_return']
    
    # add portfolio daily return to merged dataframe
    mult_df['Portfolio'] = portfolio_df.loc[portfolio_mask]['daily_return']
    
    # get covariance between the securities
    covariance = mult_df.cov() 
    # return covariance at row 0 & column 1
    cov_vs_market = covariance.iloc[0,1]
    
    # get variance for SPY
    sp_variance = mult_df['SPY'].var() 
    
    # get portfolio Beta
    beta = cov_vs_market / sp_variance
    return beta

#### Calculating Alpha

Alpha measures how well a portfolio has performed in relation to its benchmark. CAPM assumes an Alpha of 0. 

Alpha = $ r_p - r_f - \beta_p (r_m - r_f)$ 

$r_p$ represents the portfolio return, $r_f$ represents the risk-free rate of return, $\beta_p$ represents the systematic risk of a portfolio, and $r_m$ represents the market benchmark return.

In [44]:
portfolio_beta = portfolio_beta(portfolio_df, '2022-01-03', '2022-12-30') 
print("Portfolio Beta : %2.2f " % (portfolio_beta))

Portfolio Beta : 0.71 


In [48]:
print("Portfolio ROI for 2022 : %2.2f %%" % (roi_portfolio * 100))
print("Benchmark ROI for 2022 : %2.2f %%" % (sp_roi * 100))
# get portfolio alpha
portfolio_alpha = roi_portfolio - risk_free_rate - (portfolio_beta * (sp_roi - risk_free_rate))
print("Portfolio Alpha : %2.2f %%" % (portfolio_alpha * 100))

Portfolio ROI for 2022 : -2.89 %
Benchmark ROI for 2022 : -18.64 %
Portfolio Alpha : 9.87 %


#### Comments:

1. Demonstrator portfolio outperformed the market benchmark (SPY as ^GSPC trading proxy) in 2022 by 9.87% (risk adjusted). Portfolio Beta of 0.71 for 2022 appears to be an appropriate measure of risk given that portfolio lost significantly less than its benchmark.</p>

2. Introduction of a few simple market structure biases shows, at least as it relates to this Portfolio demonstrator, that share returns do not necessarily follow a random walk.</p>

3. Since interest rates increased substantially throughout 2022, and given the uncertainty about the future rate path, places some doubt on what is the appropriate risk-free rate measurement.  